In [1]:
import sys
sys.path.append('..')
import numpy as np
import cupy as cp
import dask.array as da
import jax.numpy as jnp

# LinOpStack

In [6]:
from pycsou.linop.base import LinOpStack
from pycsou.util.misc import peaks

from pycsou.linop.diff import FirstDerivative, Gradient

for xp in [np, cp]:# , da, jnp]:
    x = xp.linspace(-2.5, 2.5, 100)
    X,Y = xp.meshgrid(x,x)
    Z = peaks(X, Y)
    D1 = FirstDerivative(size=Z.size, shape=Z.shape, axis=0, kind='centered')
    D2 = FirstDerivative(size=Z.size, shape=Z.shape, axis=1, kind='centered')
    G1 = LinOpStack(D1, D2, axis=0)
    G2 = Gradient(shape=Z.shape, kind='centered')
    Z_d = D2*Z.flatten()
    print(xp.allclose(G1*Z.flatten(), G2 * Z.flatten()))
    # True
    print(xp.allclose(G1.adjoint(G1*Z.flatten()), G2.adjoint(G2 * Z.flatten())))
    # True
    G3 = LinOpStack(D1.H, D2.H, axis=1)
    print(xp.allclose(G1.adjoint(G1*Z.flatten()), (G3 * G1) * Z.flatten()))
    # True
    parG1 = LinOpStack(D1, D2, axis=0, n_jobs=-1)
    parG3 = LinOpStack(D1.H, D2.H, axis=1, n_jobs=-1)
    print(xp.allclose(G1.adjoint(G1*Z.flatten()), parG1.adjoint(parG1*Z.flatten())))
    # True
    print(xp.allclose((G3 * G1) * Z.flatten(), (parG3 * parG1) * Z.flatten()))
    # True



True
True
True
True
True
True
True
True
True
True


# BlockOperator

In [7]:
from pycsou.linop.base import BlockOperator
from pycsou.linop.diff import SecondDerivative

for xp in [np, cp]:
    Nv, Nh = 11, 21
    D2hop = SecondDerivative(size=Nv * Nh, shape=(Nv,Nh), axis=1)
    D2vop = SecondDerivative(size=Nv * Nh, shape=(Nv,Nh), axis=0)
    Dblock = BlockOperator([[D2vop, 0.5 * D2vop, - D2hop], [D2hop, 2 * D2hop, D2vop]])
    x = xp.zeros((Nv, Nh)); x[int(Nv//2), int(Nh//2)] = 1; z = xp.tile(x, (3,1)).flatten()
    print(xp.allclose(Dblock(z), xp.concatenate(((D2vop + 0.5 * D2vop - D2hop)(x.flatten()), (D2hop + 2 * D2hop + D2vop)(x.flatten())))))
    #True

True
True


# BlockDiagonalOperator

In [8]:
from pycsou.linop.base import BlockDiagonalOperator
from pycsou.linop.diff import SecondDerivative

for xp in [np, cp]:
    Nv, Nh = 11, 21
    D2hop = SecondDerivative(size=Nv * Nh, shape=(Nv,Nh), axis=1)
    D2vop = SecondDerivative(size=Nv * Nh, shape=(Nv,Nh), axis=0)
    Dblockdiag = BlockDiagonalOperator(D2vop, 0.5 * D2vop, -1 * D2hop)
    x = xp.zeros((Nv, Nh)); x[int(Nv//2), int(Nh//2)] = 1; z = xp.tile(x, (3,1)).flatten()
    print(xp.allclose(Dblockdiag(z), xp.concatenate((D2vop(x.flatten()), 0.5 * D2vop(x.flatten()), - D2hop(x.flatten())))))
    #True

True
True


# PolynomialLinearOperator

In [10]:
from pycsou.linop import DenseLinearOperator, PolynomialLinearOperator

for xp in [np, cp]:
    L = DenseLinearOperator(xp.arange(64).reshape(8,8))
    PL = PolynomialLinearOperator(LinOp=L, coeffs=[1/2 ,2, 1])
    x = xp.arange(8)
    print(xp.allclose(PL(x), x/2 + 2 * L(x) + (L**2)(x)))
    #True

True
True


# KroneckerProduct

In [11]:
from pycsou.linop.base import KroneckerProduct
from pycsou.linop.diff import SecondDerivative

for xp in [np, cp]:
    Nv, Nh = 11, 21
    D2hop = SecondDerivative(size=Nh)
    D2vop = SecondDerivative(size=Nv)
    Dkron = KroneckerProduct(D2hop, D2vop)
    x = xp.zeros((Nv, Nh)); x[int(Nv//2), int(Nh//2)] = 1
    print(xp.allclose(Dkron(x.flatten()), D2vop.apply_along_axis(D2hop.apply_along_axis(x.transpose(), axis=0).transpose(), axis=0).flatten()))
    #True


True
True


# KroneckerSum

In [6]:

from pycsou.linop.base import KroneckerProduct, KroneckerSum, DiagonalOperator

for xp in [np, cp]:
    m1=xp.linspace(0,3,5); m2=xp.linspace(-3,2,7)
    D1=DiagonalOperator(m1); 
    ExpD1=DiagonalOperator(xp.exp(m1))
    D2=DiagonalOperator(m2); ExpD2=DiagonalOperator(xp.exp(m2))
    Expkronprod=KroneckerProduct(ExpD1,ExpD2)
    Kronsum=KroneckerSum(D1,D2)
    print(xp.allclose(xp.diag(Expkronprod.todense(xp).mat), xp.exp(xp.diag(Kronsum.todense(xp).mat))))
    #True

True
True


# KhatriRaoProduct

In [2]:
from pycsou.linop.base import KhatriRaoProduct
from pycsou.linop.diff import SecondDerivative

for xp in [np, cp]:
    D1 = SecondDerivative(size=11)
    D2 = SecondDerivative(size=11)
    Dkrao = KhatriRaoProduct(D1, D2)
    x = xp.arange(11)
    print(Dkrao(x).shape)
    #(121,)
    print(D2.todense(xp).mat.transpose().flatten())
    print(xp.allclose(Dkrao(x), ((D1.todense(xp).mat * x[None, :]) @ D2.todense(xp).mat.transpose()).flatten()))
    #True

(121,)
[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -2. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1. -2.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2. -2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]
True
(121,)
[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -2. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1. -2.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 -2. -2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1